#### Table Replication  
Use predefined SQL table copies to partion and cluster new tables

In [1]:
import bq, config
from google.cloud import bigquery

In [2]:
project = "tpc-benchmarking-9432"

In [3]:
fp_schema = config.fp_schema + config.sep + "bq_h_04.sql"

In [4]:
source_dataset = "h_10000GB_01"
destination_dataset = "h_10000GB_04"

In [5]:
def create_table_remix(fp_schema, source, destination):
    with open(fp_schema, "r") as f:
        query_text = f.read()
    query_text = query_text.replace("_source_table", source)
    query_text = query_text.replace("_destination_table", destination)
    return query_text

In [6]:
query_text = create_table_remix(fp_schema=fp_schema,
                                source=source_dataset,
                                destination=destination_dataset)

In [7]:
bq.create_dataset(destination_dataset)

Dataset(DatasetReference('tpc-benchmarking-9432', 'h_10000GB_04'))

In [8]:
%%time
query_job = bq.query(query_text=query_text,
                     project=project,
                     dataset=destination_dataset,
                     dry_run=False,
                     use_cache=False)
query_job.result()

CPU times: user 287 ms, sys: 49.9 ms, total: 337 ms
Wall time: 24min 12s
